In [ ]:
#default_exp flask_client_py_readme

# Flask app from smart-on-fhir/client-py/README.md

> Getting started with Flask using client-py.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pete88b/smart-on-fhir-client-py-demo/blob/main/51_flask_client_py_readme.ipynb)

# Getting started

You can run this notebook 
- [in colab](https://colab.research.google.com/github/pete88b/smart-on-fhir-client-py-demo/blob/main/51_flask_client_py_readme.ipynb) or
    - Feel free to ignore the "Start the app", "Convert this notebook" and "Run the app locally" sections
- on your own machine
    - Please see index.ipynb and set things up to run on your own machine.

See also: https://github.com/smart-on-fhir/client-py

In [ ]:
#hide
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    !pip install -Uqq git+https://github.com/smart-on-fhir/client-py.git
    !pip install flask-ngrok

## Code and test a simple flask app

In [ ]:
#export
import logging
from fhirclient import client
from fhirclient.models.medication import Medication
from fhirclient.models.medicationrequest import MedicationRequest
from fhirclient.models.patient import Patient
from flask import Flask, request, redirect, session, url_for

## App setup

In [ ]:
#export
smart_defaults = {
    'app_id': 'my_web_app',
    'api_base': 'http://wildfhir4.aegis.net/fhir4-0-0',
#     'api_base': 'http://hapi.fhir.org/baseR4',
    'redirect_uri': 'http://localhost:8000/fhir-app/',
}

app = Flask(__name__)
app.config.from_mapping(
    # a default secret that should be overridden by instance config
    SECRET_KEY="dev"
)

True

## Helper functions

In [ ]:
#export
def _save_state(state):
    session['state'] = state

def _get_smart():
    state = session.get('state')
    if state:
        return client.FHIRClient(state=state, save_func=_save_state)
    else:
        return client.FHIRClient(settings=smart_defaults, save_func=_save_state)

def _logout():
    if 'state' in session:
        smart = _get_smart()
        smart.reset_patient()

def _reset():
    if 'state' in session:
        del session['state']

def _get_prescriptions(smart):
    bundle = MedicationRequest.where({'patient': smart.patient_id}).perform(smart.server)
    pres = [be.resource for be in bundle.entry] if bundle is not None and bundle.entry is not None else None
    if pres is not None and len(pres) > 0:
        return pres
    return None

def _get_medication_by_ref(ref, smart):
    med_id = ref.split("/")[1]
    return Medication.read(med_id, smart.server).code

def _med_name(med):
    if med.coding:
        name = next((coding.display for coding in med.coding if coding.system == 'http://www.nlm.nih.gov/research/umls/rxnorm'), None)
        if name:
            return name
    if med.text and med.text:
        return med.text
    return "Unnamed Medication(TM)"

def _get_med_name(prescription, client=None):
    if prescription.medicationCodeableConcept is not None:
        med = prescription.medicationCodeableConcept
        return _med_name(med)
    elif prescription.medicationReference is not None and client is not None:
        med = _get_medication_by_ref(prescription.medicationReference.reference, client)
        return _med_name(med)
    else:
        return 'Error: medication not found'

In [ ]:
#export
def _get_patients(smart):
    bundle = Patient.where(struct={}).perform(smart.server)
    patients = [be.resource for be in bundle.entry] if bundle is not None and bundle.entry is not None else None
    if patients is not None and len(patients) > 0:
        return patients
    return None

In [ ]:
smart = client.FHIRClient(settings=smart_defaults)
for p in _get_patients(smart): assert p.id is not None # TODO: write a better test

## Views

In [ ]:
#export
@app.route('/')
@app.route('/index.html')
def index():
    """ The app's main page.
    """
    smart = _get_smart()
    body = "<h1>Hello</h1>"
    
    if request.args.get('patient_id') is not None:
        smart.patient_id = request.args.get('patient_id')
    print('ready',smart.ready,'patient_id',smart.patient_id,'patient',smart.patient)
    
    def _format_name(patient):
        return smart.human_name(patient.name[0] if patient.name and len(patient.name) > 0 else 'Unknown')
    
    def _format_patient(patient):
        s = f'{patient.id} {_format_name(patient)} '
        if patient.birthDate is not None: s += patient.birthDate.isostring
        return s
    
    if smart.ready and smart.patient is not None: # "ready" may be true but the access token may have expired, making smart.patient = None
        name = _format_name(smart.patient)
        
        # generate simple body text
        body += "<p>You are authorized and ready to make API requests for <em>{0}</em>.</p>".format(name)
        pres = _get_prescriptions(smart)
        if pres is not None:
            body += "<p>{0} prescriptions: <ul><li>{1}</li></ul></p>".format("His" if 'male' == smart.patient.gender else "Her", '</li><li>'.join([_get_med_name(p,smart) for p in pres]))
        else:
            body += "<p>(There are no prescriptions for {0})</p>".format("him" if 'male' == smart.patient.gender else "her")
        body += """<p><a href="/logout">Change patient</a></p>"""
    else:
        auth_url = smart.authorize_url
        if auth_url is not None:
            body += """<p>Please <a href="{0}">authorize</a>.</p>""".format(auth_url)
        else:
            body += """<p>Running against a no-auth server.<br>"""
            for patient in _get_patients(smart):
                # id name dob
                body += "<hr><a href='{0}'>Select patient</a> {1}".format(
                    url_for('index', patient_id=patient.id),
                    _format_patient(patient))
#             body += '<br>'.join(resource_to_string(_get_patients(smart)))
        body += """<p><a href="/reset" style="font-size:small;">Reset</a></p>"""
    return body

In [ ]:
#export
@app.route('/fhir-app/')
def callback():
    """ OAuth2 callback interception.
    """
    smart = _get_smart()
    try:
        smart.handle_callback(request.url)
    except Exception as e:
        return """<h1>Authorization Error</h1><p>{0}</p><p><a href="/">Start over</a></p>""".format(e)
    return redirect('/')

In [ ]:
#export
@app.route('/logout')
def logout():
    _logout()
    return redirect('/')

In [ ]:
#export
@app.route('/reset')
def reset():
    _reset()
    return redirect('/')

## Start the app

In [ ]:
#export
try: from nbdev.imports import IN_NOTEBOOK
except: IN_NOTEBOOK = False
if '__main__' == __name__ and not IN_NOTEBOOK:
    logging.basicConfig(level=logging.DEBUG)
    app.run(debug=True, port=8000)

## Convert this notebook to a python module

After making changes to this notebook, we need to get nbdev to re-create the `flask_client_py_readme.py` file.

We can do this in code &darr; or from the command line with `nbdev_build_lib`

In [ ]:
if not IN_COLAB:
    from nbdev.export import notebook2script
    notebook2script('51_flask_client_py_readme.ipynb')

Converted 51_flask_client_py_readme.ipynb.


## Run the app locally

Now we can run `python smart_on_fhir_client_py_demo/flask_client_py_readme.py` from the command line and hit 
- http://localhost:8000/

## Run the app in colab

The following cell will only run if you're in colab, where you'll see output like

```
 * Serving Flask app "__main__" (lazy loading)
 ...
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): bin.equinox.io:443
...
DEBUG:urllib3.connectionpool:http://localhost:4040 "GET /api/tunnels HTTP/1.1" 200 779
 * Running on http://35c09c8e5eb5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
```

Hit the `ngrok.io` link to see the app - none of the localhost or 127.0.0.1 links will work

In [ ]:
if IN_COLAB:
    logging.basicConfig(level=logging.DEBUG)
    from flask_ngrok import run_with_ngrok
    run_with_ngrok(app)
    app.run()